# 7.2Data Transformation数据变换

1 删除重复值

In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.DataFrame({'k1':['one','two']*3+['two'],
                  'k2':[1,1,2,3,3,4,4]})

In [3]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


DataFrame方法duplicated返回的是一个boolean Series，表示一个row是否是重复的（根据前一行来判断）：

In [4]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

drop_duplicateds返回一个DataFrame，会删除重复的部分：

In [5]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


上面两种方法都默认考虑所有列；另外，我们可以指定一部分来检测重复值。假设我们只想检测'k1'列的重复值：

In [6]:
data['v1']=range(7)

In [7]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


data.drop_duplicates(['k1'])

duplicated和drop_duplicated默认保留第一次观测到的数值组合。设置keep='last'能返回最后一个：

In [8]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


#  Transforming Data Using a Function or Mapping（用函数和映射来转换数据）

In [9]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [10]:
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

用于series的map方法接受一个函数，或是一个字典，包含着映射关系，但这里有一个小问题，有些肉是大写，有些是小写。因此，我们先用str.lower把所有的值变为小写:

In [11]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [12]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


我们也可以用一个函数解决上面的问题

In [13]:
data['food'].map(lambda x:meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

使用map是一个很简便的方法，用于element-wise转换和其他一些数据清洗操作。
3 Replacing Values（替换值）

其实fillna是一个特殊换的替换操作。map可以用于修改一个object里的部分值，但是replace能提供一个更简单和更灵活的方法做到这点。下面是一个series：


In [14]:
data=pd.Series([1.,-999.,2.,-999.,-1000.,3.])

In [15]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

这里-999可能是用来表示缺失值的标识符。用NA来替代的话，用replace，会产生一个新series（除非使用inplace=True）:

In [16]:
data.replace(-999,np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

如果想要一次替换多个值，直接用一个list即可：

In [17]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

data.replace([-999, -1000], np.nan)

data.replace({-999: np.nan, -1000: 0})

注意：data.replace方法和data.str.replace方法是不同的，后者会对string进行element-wise替换。
4 Renaming Axis Indexes（重命名Axis Indexes）

像是series里的value一样，axis label也能类似地是函数或映射来转换，产生一个新的object。当然也可以设置in-place不产生新的数据：


In [18]:
data=pd.DataFrame(np.arange(12).reshape((3,4)),
                 index=['Ohio','Colorado','Network'],
                 columns=['one','two','three','four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Network,8,9,10,11


In [19]:
#与series相同，axis index有一个map方法

In [20]:
transform=lambda x:x[:4].upper()

In [21]:
transform

<function __main__.<lambda>(x)>

In [22]:
data.index

Index(['Ohio', 'Colorado', 'Network'], dtype='object')

In [23]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NETW'], dtype='object')

可以赋值给index，以in-place的方式修改DataFrame：

In [24]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NETW,8,9,10,11


如果你想要创建一个转换后的版本，而且不用修改原始的数据，可以用rename:

In [25]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
Netw,8,9,10,11


In [26]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'pekaboo'})

,one,two,pekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NETW,8,9,10,11


In [27]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NETW,8,9,10,11


5 Discretization and Binning（离散化和装箱）

连续型数据经常被离散化或分散成bins（分箱）来分析。假设你有一组数据，你想把人分到不同的年龄组里：


In [28]:
ages=[20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

我们把这些分到四个bin里，19~25, 26~35, 36~60, >60。可以用pandas里的cut：

In [29]:
bins = [18, 25, 35, 60, 100]

cats = pd.cut(ages, bins)

cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

返回的是一个特殊的Categorical object。我们看到的结果描述了pandas.cut如何得到bins。可以看作是一个string数组用来表示bin的名字，它内部包含了一个categories数组，用来记录不同类别的名字，并伴有表示ages的label（可以通过codes属性查看）：

In [30]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [31]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [32]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

这里pd.value_counts(cats)是pandas.cut后bin的数量。

这里我们注意一下区间。括号表示不包含，方括号表示包含。你可以自己设定哪一边关闭（right=False）:


In [34]:
pd.cut(ages,[18,26,36,61,100],right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

你也可以用一个list或数组给labels选项来设定bin的名字：

In [35]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [36]:
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

如果你只是给一个bins的数量来cut，而不是自己设定每个bind的范围，cut会根据最大值和最小值来计算等长的bins。比如下面我们想要做一个均匀分布的四个bins：

In [37]:
data = np.random.rand(20)

In [38]:
pd.cut(data, 4, precision=2)

[(0.044, 0.28], (0.75, 0.99], (0.52, 0.75], (0.52, 0.75], (0.044, 0.28], ..., (0.75, 0.99], (0.75, 0.99], (0.044, 0.28], (0.044, 0.28], (0.28, 0.52]]
Length: 20
Categories (4, interval[float64]): [(0.044, 0.28] < (0.28, 0.52] < (0.52, 0.75] < (0.75, 0.99]]

precision=2选项表示精确到小数点后两位。

一个近似的函数，qcut，会按照数据的分位数来分箱。取决于数据的分布，用cut通常不能保证每一个bin有一个相同数量的数据点。而qcut是按百分比来切的，所以可以得到等数量的bins：


In [39]:
data = np.random.randn(1000) # Normally distributed

In [40]:
cats = pd.qcut(data, 4) # Cut into quartiles
cats

[(0.616, 3.606], (-0.0514, 0.616], (-0.703, -0.0514], (-3.002, -0.703], (-3.002, -0.703], ..., (-0.0514, 0.616], (-0.0514, 0.616], (0.616, 3.606], (-3.002, -0.703], (-3.002, -0.703]]
Length: 1000
Categories (4, interval[float64]): [(-3.002, -0.703] < (-0.703, -0.0514] < (-0.0514, 0.616] < (0.616, 3.606]]

In [41]:
pd.value_counts(cats)

(0.616, 3.606]       250
(-0.0514, 0.616]     250
(-0.703, -0.0514]    250
(-3.002, -0.703]     250
dtype: int64

类似的，在cut中我们可以自己指定百分比：

In [43]:
cats=pd.cut(data,[0,0.1,0.5,0.9,1.])

In [44]:
cats

[(0.5, 0.9], (0.1, 0.5], NaN, NaN, NaN, ..., (0.1, 0.5], (0.1, 0.5], (0.5, 0.9], NaN, NaN]
Length: 1000
Categories (4, interval[float64]): [(0.0, 0.1] < (0.1, 0.5] < (0.5, 0.9] < (0.9, 1.0]]

在之后的章节我们还会用到cut和qcut，这些离散函数对于量化和群聚分析很有用。
6 Detecting and Filtering Outliers（检测和过滤异常值）

过滤或转换异常值是数组操作的一个重头戏。下面的DataFrame有正态分布的数据：


In [45]:
data=pd.DataFrame(np.random.randn(1000,4))

In [46]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.041688,-0.039351,0.069520,0.003459
std,1.003989,0.980138,0.949156,1.021245
min,-2.973638,-2.995868,-2.898253,-4.068079
25%,-0.621742,-0.701841,-0.587263,-0.640444
50%,0.092714,-0.019064,0.065003,-0.029320
75%,0.804403,0.617786,0.696635,0.698121
max,2.958871,3.079776,3.303698,2.956351


In [47]:
data.head()

,0,1,2,3
0,0.575209,-0.882966,-0.227736,1.745781
1,-0.734843,-0.111194,0.433636,-1.516025
2,0.148840,-1.017897,-2.025813,1.579666
3,0.879632,1.451461,1.828063,-0.744012
4,-0.057241,-0.776037,0.523520,1.782563


In [48]:
col=data[2]

In [49]:
col.head()

0   -0.227736
1    0.433636
2   -2.025813
3    1.828063
4    0.523520
Name: 2, dtype: float64

In [51]:
col[np.abs(col)>3]

178    3.303698
452    3.243909
Name: 2, dtype: float64

In [52]:
data[(np.abs(data)>3)].head()

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [53]:
data[(np.abs(data)>3).any(1)]

,0,1,2,3
178,-0.570579,0.803455,3.303698,-0.753032
195,1.815598,0.882354,-0.909872,-4.068079
452,-1.418525,1.056733,3.243909,0.174815
753,-0.105398,0.042028,-0.222196,-3.379026
759,0.225322,-0.592097,-1.745976,-3.000294
970,-0.102357,3.079776,0.558683,-0.131581


In [54]:
data[np.abs(data)>3]=np.sign(data)*3

In [55]:
data[21:23]

,0,1,2,3
21,-2.039149,-0.795561,1.155970,-0.378305
22,0.114157,0.240349,0.301972,-0.878108


In [56]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.041688,-0.039430,0.068972,0.004906
std,1.003989,0.979887,0.947382,1.016342
min,-2.973638,-2.995868,-2.898253,-3.000000
25%,-0.621742,-0.701841,-0.587263,-0.640444
50%,0.092714,-0.019064,0.065003,-0.029320
75%,0.804403,0.617786,0.696635,0.698121
max,2.958871,3.000000,3.000000,2.956351


In [57]:
#np.sign(data)会根据值的正负号来得到1或-1：

In [58]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,-1.0,1.0
1,-1.0,-1.0,1.0,-1.0
2,1.0,-1.0,-1.0,1.0
3,1.0,1.0,1.0,-1.0
4,-1.0,-1.0,1.0,1.0


7 Permutation and Random Sampling（排列和随机采样）

排列（随机排序）一个series或DataFrame中的row，用numpy.random.permutation函数很容易就能做到。调用permutation的时候设定好你想要进行排列的axis，会产生一个整数数组表示新的顺序：


In [59]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [62]:
sampler=np.random.permutation(5)

In [63]:
sampler

array([2, 3, 4, 0, 1])

In [64]:
#这个数组能被用在基于iloc上的indexing或take函数：

In [65]:
df.take(sampler)

,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
0,0,1,2,3
1,4,5,6,7


为了选中一个随机的子集，而且没有代替功能(既不影响原来的值，返回一个新的series或DataFrame)，可以用sample方法：

In [66]:
df.sample(n=3)

,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


如果想要生成的样本带有替代功能(即允许重复)，给sample中设定replace=True:

In [67]:
choices = pd.Series([5, 7, -1, 6, 4])

draws = choices.sample(n=10, replace=True)

draws

2   -1
4    4
0    5
3    6
2   -1
1    7
1    7
2   -1
2   -1
3    6
dtype: int64

8 Computing Indicator/Dummy Variables（计算指示器/虚拟变量）

    Dummy Variables：虚拟变量，又称虚设变量、名义变量或哑变量,用以反映质的属性的一个人工变量,是量化了的自变量,通常取值为0或1。

另一种在统计模型上的转换或机器学习应用是把一个categorical variable(类别变量)变为一个dummy or indicator matrix（虚拟或指示器矩阵）。如果DataFrame中的一列有k个不同的值，我们可以用一个矩阵或DataFrame用k列来表示，1或0。pandas有一个get_dummies函数实现这个工作，当然，你自己设计一个其实也不难。这里举个例子：


In [68]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [69]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


在一些情况里，如果我们想要给column加一个prefix， 可以用data.get_dummies里的prefix参数来实现：

In [70]:
dummies = pd.get_dummies(df['key'], prefix='key')

In [71]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [72]:
mnames = ['movie_id', 'title', 'genres']

In [73]:
movies = pd.read_table('../datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames, engine='python')
movies[:10]

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/movielens/movies.dat'

In [74]:
all_genres = []

for x in movies.genres:
    all_genres.extend(x.split('|'))
    
genres = pd.unique(all_genres)
genres

NameError: name 'movies' is not defined